In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Clone NHÁNH convnext-v2-dev
if not os.path.exists('/content/FR_Photometric_Stereo'):
    !git clone -b convnext-v2-dev https://github.com/minh4923/FR_Photometric_Stereo.git

# 3. Di chuyển vào thư mục dự án
%cd /content/FR_Photometric_Stereo

# 4. Cài đặt thư viện
!pip install -q albumentations==1.3.1 timm


Mounted at /content/drive
Cloning into 'FR_Photometric_Stereo'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 142 (delta 48), reused 132 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (142/142), 10.79 MiB | 33.09 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/FR_Photometric_Stereo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
%cd /content/FR_Photometric_Stereo
import warnings
warnings.filterwarnings("ignore")
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import torch
import pandas as pd

from going_modular.dataloader.multitask import create_concatv2_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.model.ConcatMTLFaceRecognition import ConcatMTLFaceRecognitionV2
from going_modular.loss.ConcatMultiTaskLoss import ConcatMultiTaskLoss
from going_modular.train_eval.concat_train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.ExperimentManager import ExperimentManager

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

# --- TÊN THÍ NGHIỆM ---
EXPERIMENT_NAME = "Concat_ALBEDO_NORMAL_Shuffle"

CONFIGURATION = {
    'note': EXPERIMENT_NAME,
    'type': 'concat_v2',

    # CÔNG TẮC SHUFFLE
    'use_sampler': False,

    # Thư mục & Checkpoint
    'dataset_dir': '/content/drive/MyDrive/Photometric_DB_Full/',
    'checkpoint_1': '/content/drive/MyDrive/Photometric_DB_Full/experiments/Single_ALBEDO_Shuffle/checkpoints/best_model.pth',
    'checkpoint_2': '/content/drive/MyDrive/Photometric_DB_Full/experiments/Single_NORMALMAP_Shuffle/checkpoints/best_model.pth',

    'device': device,
    'epochs': 119,
    'num_workers': 2,
    'batch_size': 16,
    'image_size': 112,
    'base_lr': 1e-4,
    'backbone': 'miresnet18',
    'num_classes': None,


    'loss_gender_weight': 1.0,      'loss_da_gender_weight': 0.1,
    'loss_emotion_weight': 0.5,     'loss_da_emotion_weight': 0.1,
    'loss_pose_weight': 1.0,        'loss_da_pose_weight': 0.1,
    'loss_spectacles_weight': 0.5,  'loss_da_spectacles_weight': 0.1,
    'loss_facial_hair_weight': 0.5, 'loss_da_facial_hair_weight': 0.1,
}

# --- 1. MANAGER ---
manager = ExperimentManager(CONFIGURATION)
manager.log_text(f"MODE: {EXPERIMENT_NAME} | Shuffle: {not CONFIGURATION['use_sampler']}")

# --- 2. DATASET ---
train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'dataset/train_split.csv')
if not os.path.exists(train_csv_path): train_csv_path = os.path.join(CONFIGURATION['dataset_dir'], 'train_split.csv')
df_temp = pd.read_csv(train_csv_path)
CONFIGURATION['num_classes'] = df_temp['id'].max() + 1
manager.log_text(f"Num Classes: {CONFIGURATION['num_classes']}")

train_transform = A.Compose([
    RandomResizedCropRect(CONFIGURATION['image_size']),
    GaussianNoise(p=0.2),
], additional_targets={'image2': 'image'})

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={'image2': 'image'})

# Load Data (Shuffle/Sampler tự động theo config)
train_dataloader, test_dataloader = create_concatv2_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)

# --- 3. MODEL SETUP ---
# Load Backbone 1 (Albedo)
checkpoint_1 = torch.load(CONFIGURATION['checkpoint_1'], map_location=device, weights_only=False)

mtl_backbone1 = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
# Fix lỗi key nếu có 'module.' prefix
state_dict_1 = {k.replace('module.', ''): v for k, v in checkpoint_1['model_state_dict'].items()}
mtl_backbone1.load_state_dict(state_dict_1, strict=False)

# Load Backbone 2 (Normal)
checkpoint_2 = torch.load(CONFIGURATION['checkpoint_2'], map_location=device, weights_only=False)

mtl_backbone2 = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
state_dict_2 = {k.replace('module.', ''): v for k, v in checkpoint_2['model_state_dict'].items()}
mtl_backbone2.load_state_dict(state_dict_2, strict=False)

# Combine Model
model = ConcatMTLFaceRecognitionV2(mtl_backbone1, mtl_backbone2, CONFIGURATION['num_classes'])

# Freeze Backbones (Chỉ train phần đầu Concat)
for param in model.mtl_backbone1.parameters(): param.requires_grad = False
for param in model.mtl_backbone2.parameters(): param.requires_grad = False

criterion = ConcatMultiTaskLoss(train_csv_path, CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)

# --- 4. CHECKPOINT & EARLY STOP ---
ckpt_saver = ModelCheckpoint(
    output_dir=manager.ckpt_dir,
    mode='max',
    best_metric_name='auc_id_cosine'
)

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc'], patience=1000, mode='max', verbose=0,
    save_dir=manager.ckpt_dir, start_from_epoch=0
)

# --- 5. FIT ---
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    early_stopping=early_stopping,
    model_checkpoint=ckpt_saver,
    existing_manager=manager
)

/content/FR_Photometric_Stereo
KHOI TAO THI NGHIEM: Concat_ALBEDO_NORMAL_Shuffle
Luu tru tai: /content/drive/MyDrive/Photometric_DB_Full/experiments/Concat_ALBEDO_NORMAL_Shuffle
Thoi gian: 2026-01-10 10:32:53
--------------------------------------------------
MODE: Concat_ALBEDO_NORMAL_Shuffle | Shuffle: True
Num Classes: 2232
>>> ConcatV2Loader: MODE = RANDOM SHUFFLE
BAT DAU CONCAT TRAINING: Concat_ALBEDO_NORMAL_Shuffle

--- Epoch 1/119 ---
Ep 1:
╒══════════════════╤════════════╤═════════════════════╕
│ Metric           │      Train │ Test                │
╞══════════════════╪════════════╪═════════════════════╡
│ loss             │ 26.3038    │ -                   │
├──────────────────┼────────────┼─────────────────────┤
│ loss_id          │ 26.0792    │ -                   │
├──────────────────┼────────────┼─────────────────────┤
│ loss_gender      │  0.0335484 │ 0.03544724898205863 │
├──────────────────┼────────────┼─────────────────────┤
│ loss_emotion     │  0.134922  │ 0.17678268